In [36]:
import os, sys, json, csv
import tweepy as tw
import pandas as pd

from oauth2client.client import GoogleCredentials
from google.colab import drive

gdrive_path = '/content/gdrive'
full_path = gdrive_path + '/My Drive/Descartes/MyVMI/Text/Projet/data/tweets/'
drive.mount(gdrive_path)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
consumer_key= 'NNpInMgJZx1GnasnSToz6HzHH'
consumer_secret= 'v1SrwervXFxPWbM4StxlQ7MWF8YgBvuSHwtSXcis0Ot0EulAxd'
access_token= '570979551-RHqKWNFFXqXl1oW0EDH2aVx1elkaijLCsvlmh9Va'
access_token_secret= 'c8AsFii2JSvol9iEIGUNWgHlY8NNZ0KhUxfl80BZFKMpe'

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

def read_queries_file(file):
  with open(file, 'r') as f:
    queries = f.read().split(',')
  f.close()

  return queries

def searchTweets(query,date_since,limit=1000,language='en'):
  
  tweets = tw.Cursor(api.search, tweet_mode='extended',
                q=query,
                lang=language,
                since=date_since).items(limit)

  #searched_tweets = [{'text':status._json['retweeted_status']['full_text'],'date':status._json['retweeted_status']['created_at']} for status in tweets if 'retweeted_status' in status._json else {'text':status._json['full_text'],'date':status._json['created_at']} for status in tweets ]
  searched_tweets = {}

  for status in tweets:
    if 'retweeted_status' in status._json:
      searched_tweets[status._json['retweeted_status']['id_str']] = {'text':status._json['retweeted_status']['full_text'],'date':status._json['retweeted_status']['created_at']}

    else:
      searched_tweets[status._json['id_str']] = {'text':status._json['full_text'],'date':status._json['created_at']}

  return searched_tweets


def exportTweetsJSON(tweets_list,file='tweets.json'):
  with open(file, 'w') as f:
    json.dump(tweets_list, f)
  f.close()

def exportTweetsCSV(tweets_list,file='tweets.csv'):
  pd.DataFrame(tweets_list).to_csv(file, index=False)
  pass

def exportTweetsText(tweets_list,file='tweets.txt'):
  with open(file, 'wb') as f:
    for item in tweets_list:
      f.write(tweets_list[item]['text'].encode(sys.stdout.encoding, errors='replace'))
  f.close()

#query_list = read_queries_file(full_path+'../tweets_queries.txt')
query_list = ['Brexit']
date_start_search = '2018-12-31'
for query in query_list:
  alltweets = searchTweets(query,date_start_search,language='en')
  exportTweetsJSON(alltweets,full_path+'english/json/'+query+'.json')
  exportTweetsCSV(alltweets,full_path+'english/csv/'+query+'.csv')
  exportTweetsText(alltweets,full_path+'english/txt/'+query.strip(' ')+'.txt')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [38]:
import spacy
import sys
from spacy.tokens import Doc
from spacy.vocab import Vocab
from spacy.language import Language
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English

from oauth2client.client import GoogleCredentials
from google.colab import drive

gdrive_path = '/content/gdrive'
full_path = gdrive_path + '/My Drive/Descartes/MyVMI/Text/Projet/data/tweets/'
drive.mount(gdrive_path)

#Creation du motizer
nlp = spacy.load("en_core_web_sm")
tokenizer = nlp.Defaults.create_tokenizer(nlp)

print("==========================================================================================================")
print("Bienvenu dans le programme de décomposition et de recherche de sens de l'équipe les Seigneurs du temps !\n")
print("==========================================================================================================")

#Pour utiliser ce code il faut lancer le fichier avec cette commande : python start.py text.txt où text.txt est le fichier de data.

def open_file(road):
    with open(road) as file:
        text = file.read()
    return text

def research(text):
    #Tableau contenant le sujet, l'action et la date
    result=[]
    test=[]
    id=0
    #Application du NLP sur le text
    doc = nlp(text)

    for mot in doc:
      if not mot.text.startswith('@'):
        taille_result=len(result)-1
        #print("#################### Données du mot ###########################")
        #print(mot.text, mot.dep_, mot.head.text, mot.head.pos_,[enfant for enfant in mot.children])
        stop=0
        if("ROOT"==mot.dep_):
            action = mot.text
            for enfant in mot.children:
                if(stop==1):
                    break
                if(result[len(result)-1][0]!=enfant.text):
                    if(enfant.dep_!="aux" and enfant.dep_!="punct" and enfant.dep_!="advc1"):
                        for token_test in enfant.children:
                            test.append(enfant.text)
                        if(test != []):
                            first_step=1
                            for petit_enfant in enfant.children:
                                if(petit_enfant.dep_=="poss" or petit_enfant.dep_=="aux"):
                                    action +=" "+petit_enfant.text+" "+enfant.text
                                elif(first_step):
                                    action += " "+enfant.text+" "+petit_enfant.text
                                else:
                                    action += " "+petit_enfant.text
                                first_step=0
                                if(petit_enfant.dep_=="pobj" or petit_enfant.dep_=="acomp"):
                                    stop=1;
                        else:
                            action += " "+enfant.text
                        if(enfant.dep_=="pobj" or enfant.dep_=="acomp" or enfant.dep_=="dobj" or enfant.dep_=="advc1"):
                            break
            result[len(result)-1].append(action)
        if(mot.dep_=="nsubj"):
            result.append([mot.text])
        #print(mot.text,' | ',mot.dep_)


    for entity in doc.ents:
        if(entity.label_=="DATE"):
            result[id].append(entity.text)
            id+=1

    return result

def affich_result(result):
    print("=======================================================================================================")
    print("Tableaux finaux :")
    print("=======================================================================================================")
    print("Qui ? | Quoi ? | Quand ?")
    for line in result:
        print(line)
    print("=======================================================================================================")

    """x = PrettyTable(["Qui ?","Quoi ?","Quand ?"])
    for line in result:
      x.add_row(line)
    print(x.get_string(title="Tableau final :"))"""


text = open_file('/content/gdrive/My Drive/Brexit.txt')
result = research(text)
affich_result(result)


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Bienvenu dans le programme de décomposition et de recherche de sens de l'équipe les Seigneurs du temps !

Tableaux finaux :
Qui ? | Quoi ? | Quand ?
['one', 'unsurprised Why is see that institutions BrexitFestival as', '200bn']
['institutions', 'the end of the year']
['Expo', 'increases tourism International', '2018']
['It', 'is clear pretty', '47 years']
['Festival', 'https://t.co/wr5jUhWofsBrexiteers erupt in reach', 'February 1st']
['cost', 'says', 'weeks']
['analysis', 'https://t.co/UbIzmCS0wk puts it billion', 'five years']
['it', 'cost of belonging', '_', '2016']
['Eu', 'is unemployment high', 'today']
['I', 'enjoy conversing with euro', 'many years']
['they', 'vaunted jobs same story- , pay .. yet eu the to be be and end', '1955- back way doctrine eu', 'the last 3 years']
['Brexit', '2016  ']
['commentators', 'arguing Brexit & amp implode that sooner